# Selector Group Chat

{py:class}`~autogen_agentchat.teams.SelectorGroupChat`는 참가자들이 차례로 다른 모든 멤버에게 메시지를 방송하는 팀을 구현합니다. 생성 모델(예: LLM)이 공유된 컨텍스트를 기반으로 다음 발언자를 선택하여 동적이고 컨텍스트 인식 협업을 가능하게 합니다.

주요 기능:

- 모델 기반 발언자 선택
- 구성 가능한 참가자 역할 및 설명
- 동일한 발언자의 연속 발언 방지 (선택 사항)
- 사용자 정의 선택 프롬프트
- 기본 모델 기반 선택을 재정의하는 사용자 정의 선택 함수
- 모델을 사용하여 선택할 에이전트 세트를 좁히는 사용자 정의 후보 함수

```{note}
{py:class}`~autogen_agentchat.teams.SelectorGroupChat`는 고수준 API입니다. 더 많은 제어와 커스터마이징이 필요한 경우, 코어 API 문서의 [그룹 채팅 패턴](../core-user-guide/design-patterns/group-chat.ipynb)을 참조하여 자체 그룹 채팅 로직을 구현하세요.
```

## 어떻게 작동하나요?

{py:class}`~autogen_agentchat.teams.SelectorGroupChat`는 {py:class}`~autogen_agentchat.teams.RoundRobinGroupChat`와 유사한 그룹 채팅이지만, 모델 기반 다음 발언자 선택 메커니즘을 사용합니다.
팀이 {py:meth}`~autogen_agentchat.teams.BaseGroupChat.run` 또는 {py:meth}`~autogen_agentchat.teams.BaseGroupChat.run_stream`을 통해 작업을 받으면, 다음 단계가 실행됩니다:

1. 팀은 대화 기록과 참가자들의 {py:attr}`~autogen_agentchat.base.ChatAgent.name` 및 {py:attr}`~autogen_agentchat.base.ChatAgent.description` 속성을 포함한 현재 대화 컨텍스트를 분석하여 모델을 사용해 다음 발언자를 결정합니다. 기본적으로 팀은 사용 가능한 에이전트가 하나뿐이 아닌 이상 동일한 발언자를 연속으로 선택하지 않습니다. 이는 `allow_repeated_speaker=True`로 설정하여 변경할 수 있습니다. 사용자 정의 선택 함수를 제공하여 모델을 재정의할 수도 있습니다.
2. 팀은 선택된 발언자 에이전트에게 응답을 요청하고, 이는 다른 모든 참가자에게 **방송**됩니다.
3. 대화가 끝나야 하는지 종료 조건을 확인하고, 그렇지 않다면 1단계부터 과정을 반복합니다.
4. 대화가 끝나면 팀은 이 작업의 대화 기록을 포함한 {py:class}`~autogen_agentchat.base.TaskResult`를 반환합니다.

팀이 작업을 완료하면 대화 컨텍스트는 팀과 모든 참가자에게 유지되므로, 다음 작업은 이전 대화 컨텍스트에서 계속할 수 있습니다.
{py:meth}`~autogen_agentchat.teams.BaseGroupChat.reset`을 호출하여 대화 컨텍스트를 재설정할 수 있습니다.

이 섹션에서는 웹 검색 및 데이터 분석 작업의 간단한 예제로 {py:class}`~autogen_agentchat.teams.SelectorGroupChat`를 사용하는 방법을 보여드리겠습니다.

## 예제: 웹 검색/분석

In [18]:
from typing import List, Sequence

from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.messages import BaseAgentEvent, BaseChatMessage
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console

### 에이전트

![선택자 그룹 채팅](images/selector-group-chat.svg)

이 시스템은 세 개의 전문 에이전트를 사용합니다:

- **계획 에이전트**: 복잡한 작업을 관리 가능한 하위 작업으로 나누는 전략적 조정자입니다.
- **웹 검색 에이전트**: `search_web_tool`과 인터페이스하는 정보 검색 전문가입니다.
- **데이터 분석 에이전트**: `percentage_change_tool`을 갖춘 계산 수행 전문 에이전트입니다.

`search_web_tool`과 `percentage_change_tool`은 에이전트가 작업을 수행하는 데 사용할 수 있는 외부 도구입니다.

In [19]:
# Note: This example uses mock tools instead of real APIs for demonstration purposes
def search_web_tool(query: str) -> str:
    if "2006-2007" in query:
        return """2006-2007 시즌 마이애미 히트 선수들이 득점한 총 득점은 다음과 같습니다:
        우도니스 하슬렘: 844점
        드웨인 웨이드: 1397점
        제임스 포시: 550점
        ...
        """
    elif "2007-2008" in query:
        return "2007-2008시즌 마이애미 히트에서 드웨인 웨이드가 기록한 총 리바운드 수는 214개입니다."
    elif "2008-2009" in query:
        return "2008-2009 시즌 마이애미 히트에서 드웨인 웨이드가 기록한 총 리바운드 수는 398개입니다."
    return "No data found."


def percentage_change_tool(start: float, end: float) -> float:
    return ((end - start) / start) * 100

{py:class}`~autogen_agentchat.agents.AssistantAgent` 클래스를 사용하여 전문 에이전트를 생성해 보겠습니다.
에이전트의 {py:attr}`~autogen_agentchat.base.ChatAgent.name` 및 {py:attr}`~autogen_agentchat.base.ChatAgent.description` 속성이 모델에서 다음 발언자를 결정하는 데 사용되므로, 의미있는 이름과 설명을 제공하는 것이 좋습니다.

In [20]:
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient

from dotenv import load_dotenv
import os

load_dotenv(override=True)

api_version = os.getenv("AZURE_OPENAI_API_VERSION")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
azure_openai_chat_completion_client = AzureOpenAIChatCompletionClient(
            model=deployment_name,
            azure_endpoint=azure_endpoint,
            api_version=api_version,
            api_key=api_key,
        )

model_client = azure_openai_chat_completion_client


planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        WebSearchAgent: Searches for information
        DataAnalystAgent: Performs calculations

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)

web_search_agent = AssistantAgent(
    "WebSearchAgent",
    description="An agent for searching information on the web.",
    tools=[search_web_tool],
    model_client=model_client,
    system_message="""
    You are a web search agent.
    Your only tool is search_tool - use it to find information.
    You make only one search call at a time.
    Once you have the results, you never do calculations based on them.
    """,
)

data_analyst_agent = AssistantAgent(
    "DataAnalystAgent",
    description="An agent for performing calculations.",
    model_client=model_client,
    tools=[percentage_change_tool],
    system_message="""
    You are a data analyst.
    Given the tasks you have been assigned, you should analyze the data and provide results using the tools provided.
    If you have not seen the data, ask for it.
    """,
)

```{note}
기본적으로 {py:class}`~autogen_agentchat.agents.AssistantAgent`는 도구 출력을 응답으로 반환합니다. 도구가 자연어 형식으로 잘 구성된 문자열을 반환하지 않는 경우, 에이전트를 생성할 때 `reflect_on_tool_use=True`를 설정하여 에이전트 내에서 반성 단계를 추가할 수 있습니다. 이를 통해 에이전트가 도구 출력을 반성하고 자연어 응답을 제공할 수 있습니다.
```

### 워크플로우

1. 작업이 {py:class}`~autogen_agentchat.teams.SelectorGroupChat`에 의해 수신되면, 에이전트 설명을 기반으로 초기 작업을 처리할 가장 적절한 에이전트(일반적으로 계획 에이전트)를 선택합니다.

2. **계획 에이전트**는 작업을 분석하고 하위 작업으로 나누어 다음 형식을 사용하여 각각을 가장 적절한 에이전트에 할당합니다:
   `<에이전트> : <작업>`

3. 대화 컨텍스트와 에이전트 설명을 기반으로 {py:class}`~autogen_agent.teams.SelectorGroupChat` 관리자가 할당된 하위 작업을 처리할 다음 에이전트를 동적으로 선택합니다.

4. **웹 검색 에이전트**는 한 번에 하나씩 검색을 수행하고 결과를 공유 대화 기록에 저장합니다.

5. **데이터 분석자**는 선택되었을 때 사용 가능한 계산 도구를 사용하여 수집된 정보를 처리합니다.

6. 다음 중 하나가 발생할 때까지 에이전트가 동적으로 선택되면서 워크플로우가 계속됩니다:
   - 계획 에이전트가 모든 하위 작업이 완료되었다고 판단하고 "TERMINATE"를 보냄
   - 대안적인 종료 조건이 충족됨 (예: 최대 메시지 수)

에이전트를 정의할 때 다음에 선택할 에이전트를 결정하는 데 사용되므로 도움이 되는 {py:attr}`~autogen_agentchat.base.ChatAgent.description`을 포함해야 합니다.

### 종료 조건

두 가지 종료 조건을 사용해 보겠습니다:
계획 에이전트가 "TERMINATE"를 보낼 때 대화를 종료하는 {py:class}`~autogen_agentchat.conditions.TextMentionTermination`과 무한 루프를 방지하기 위해 대화를 25개 메시지로 제한하는 {py:class}`~autogen_agentchat.conditions.MaxMessageTermination`입니다.

In [21]:
text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination

### 선택자 프롬프트

{py:class}`~autogen_agentchat.teams.SelectorGroupChat`는 대화 컨텍스트를 기반으로 다음 발언자를 선택하기 위해 모델을 사용합니다.
워크플로우와 적절히 정렬하기 위해 사용자 정의 선택자 프롬프트를 사용하겠습니다.

In [6]:
selector_prompt = """Select an agent to perform task.

{roles}

Current conversation context:
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure the planner agent has assigned tasks before other agents start working.
Only select one agent.
"""

선택자 프롬프트에서 사용할 수 있는 문자열 변수들:
- `{participants}`: 선택 후보의 이름들. 형식은 `["<이름1>", "<이름2>", ...]`입니다.
- `{roles}`: 후보 에이전트의 이름과 설명을 줄바꿈으로 구분한 목록. 각 줄의 형식은: `"<이름> : <설명>"`입니다.
- `{history}`: 이름과 메시지 내용의 이중 줄바꿈으로 구분하여 형식화된 대화 기록. 각 메시지의 형식은: `"<이름> : <메시지 내용>"`입니다.

```{tip}
선택자 프롬프트에서 모델에 너무 많은 지시사항을 주지 않도록 하세요.

너무 많다는 것은 무엇인가요? 사용하는 모델의 기능에 따라 다릅니다.
GPT-4o와 동급 모델의 경우, 각 발언자가 언제 선택되어야 하는지에 대한 조건이 있는 선택자 프롬프트를 사용할 수 있습니다.
Phi-4와 같은 더 작은 모델의 경우, 이 예제에서 사용된 것과 같이 선택자 프롬프트를 가능한 한 간단하게 유지해야 합니다.

일반적으로 각 에이전트에 대해 여러 조건을 작성하고 있다면, 사용자 정의 선택 함수를 사용하거나 작업을 별도의 에이전트나 팀에서 처리할 더 작은 순차적 작업으로 나누는 것을 고려해야 한다는 신호입니다.
```

### 팀 실행

에이전트, 종료 조건, 사용자 정의 선택자 프롬프트로 팀을 생성해 보겠습니다.

In [22]:
team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,  # Allow an agent to speak multiple turns in a row.
)

이제 NBA 선수에 대한 정보를 찾는 작업으로 팀을 실행합니다.

In [23]:
task = "2006-2007 시즌에 가장 많은 득점을 기록한 마이애미 히트 선수는 누구이며, 2007-2008 시즌과 2008-2009 시즌의 총 리바운드 변화율은 얼마였나요?"

In [24]:
# Use asyncio.run(...) if you are running this in a script.
await Console(team.run_stream(task=task))

---------- TextMessage (user) ----------
2006-2007 시즌에 가장 많은 득점을 기록한 마이애미 히트 선수는 누구이며, 2007-2008 시즌과 2008-2009 시즌의 총 리바운드 변화율은 얼마였나요?
---------- ToolCallRequestEvent (WebSearchAgent) ----------
[FunctionCall(id='call_s3JZdoUnxpsAgqhdQZtxRATp', arguments='{"query": "2006-2007 season highest scorer Miami Heat"}', name='search_web_tool'), FunctionCall(id='call_PzqPTihErzaAPvzQTmelxrfz', arguments='{"query": "Miami Heat total rebounds 2007-2008 season"}', name='search_web_tool'), FunctionCall(id='call_roEJYtm4bmh6U1qNGal7W5UG', arguments='{"query": "Miami Heat total rebounds 2008-2009 season"}', name='search_web_tool')]
---------- ToolCallExecutionEvent (WebSearchAgent) ----------
[FunctionExecutionResult(content='2006-2007 시즌 마이애미 히트 선수들이 득점한 총 득점은 다음과 같습니다:\n        우도니스 하슬렘: 844점\n        드웨인 웨이드: 1397점\n        제임스 포시: 550점\n        ...\n        ', name='search_web_tool', call_id='call_s3JZdoUnxpsAgqhdQZtxRATp', is_error=False), FunctionExecutionResult(content='2007-2008시즌 마이애미 히트에서 드웨

TaskResult(messages=[TextMessage(id='dce8fd11-2e22-4fae-9d20-a2f391def907', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 15, 6, 6, 16, 779263, tzinfo=datetime.timezone.utc), content='2006-2007 시즌에 가장 많은 득점을 기록한 마이애미 히트 선수는 누구이며, 2007-2008 시즌과 2008-2009 시즌의 총 리바운드 변화율은 얼마였나요?', type='TextMessage'), ToolCallRequestEvent(id='15760b41-dc03-4f58-9174-27fc53846b82', source='WebSearchAgent', models_usage=RequestUsage(prompt_tokens=149, completion_tokens=91), metadata={}, created_at=datetime.datetime(2025, 7, 15, 6, 6, 19, 139031, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_s3JZdoUnxpsAgqhdQZtxRATp', arguments='{"query": "2006-2007 season highest scorer Miami Heat"}', name='search_web_tool'), FunctionCall(id='call_PzqPTihErzaAPvzQTmelxrfz', arguments='{"query": "Miami Heat total rebounds 2007-2008 season"}', name='search_web_tool'), FunctionCall(id='call_roEJYtm4bmh6U1qNGal7W5UG', arguments='{"query": "Miami Heat total rebounds 2008-200

보시다시피, 웹 검색 에이전트가 필요한 검색을 수행하고 데이터 분석 에이전트가 필요한 계산을 완료한 후, 드웨인 웨이드가 2006-2007 시즌에 마이애미 히트에서 가장 많은 득점을 한 선수이며, 2007-2008 시즌과 2008-2009 시즌 사이 그의 총 리바운드 변화율이 85.98%라는 것을 발견했습니다!

## 사용자 정의 선택자 함수

종종 선택 과정에 대한 더 나은 제어를 원합니다.
이를 위해 기본 모델 기반 선택을 재정의하는 사용자 정의 선택자 함수로 `selector_func` 인수를 설정할 수 있습니다.
이를 통해 더 복잡한 선택 로직과 상태 기반 전환을 구현할 수 있습니다.

예를 들어, 전문 에이전트 이후에 진행 상황을 확인하기 위해 계획 에이전트가 즉시 발언하기를 원합니다.

```{note}
사용자 정의 선택자 함수에서 `None`을 반환하면 기본 모델 기반 선택을 사용합니다.
``` 

```{note}
사용자 정의 선택자 함수는 SelectorGroupChat 팀에서 `.dump_component()`가 호출될 때 [직렬화](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/serialize-components.html)되지 않습니다. 사용자 정의 선택자 함수가 있는 팀 구성을 직렬화해야 하는 경우, 사용자 정의 워크플로우와 직렬화 로직을 구현하는 것을 고려하세요.
```

In [ ]:
def selector_func(messages: Sequence[BaseAgentEvent | BaseChatMessage]) -> str | None:
    if messages[-1].source != planning_agent.name:
        return planning_agent.name
    return None


# Reset the previous team and run the chat again with the selector function.
await team.reset()
team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,
    selector_func=selector_func,
)

await Console(team.run_stream(task=task))

---------- TextMessage (user) ----------
2006-2007 시즌에 가장 많은 득점을 기록한 마이애미 히트 선수는 누구이며, 2007-2008 시즌과 2008-2009 시즌의 총 리바운드 변화율은 얼마였나요?
---------- TextMessage (PlanningAgent) ----------
1. WebSearchAgent : 2006-2007 시즌에 마이애미 히트에서 가장 많은 득점을 기록한 선수의 정보를 찾아주세요.
2. WebSearchAgent : 2007-2008 시즌과 2008-2009 시즌의 마이애미 히트 선수들의 총 리바운드 통계를 찾아주세요.
3. DataAnalystAgent : 2007-2008 시즌과 2008-2009 시즌의 마이애미 히트 총 리바운드 수의 변화율을 계산해주세요.
---------- ToolCallRequestEvent (WebSearchAgent) ----------
[FunctionCall(id='call_zJpBlqXJUbQ3Svt18nJPcdgQ', arguments='{"query": "2006-2007 시즌 마이애미 히트 최고 득점 선수"}', name='search_web_tool'), FunctionCall(id='call_WAkAmmmtk35DmzbQUFIbwXDK', arguments='{"query": "2007-2008 시즌 마이애미 히트 선수 총 리바운드 통계"}', name='search_web_tool'), FunctionCall(id='call_Q0nUWbd5HCqIrA5pOmypZS8C', arguments='{"query": "2008-2009 시즌 마이애미 히트 선수 총 리바운드 통계"}', name='search_web_tool')]
---------- ToolCallExecutionEvent (WebSearchAgent) ----------
[FunctionExecutionResult(content='2006-2007 시즌 마이애미 히트 선수들이 득점

TaskResult(messages=[TextMessage(id='f32c37ba-bf0e-476b-a054-3713f0f75d4f', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 15, 6, 7, 19, 937030, tzinfo=datetime.timezone.utc), content='2006-2007 시즌에 가장 많은 득점을 기록한 마이애미 히트 선수는 누구이며, 2007-2008 시즌과 2008-2009 시즌의 총 리바운드 변화율은 얼마였나요?', type='TextMessage'), TextMessage(id='0f494a0f-f22b-40f3-be28-ad576f526639', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=168, completion_tokens=120), metadata={}, created_at=datetime.datetime(2025, 7, 15, 6, 7, 21, 286226, tzinfo=datetime.timezone.utc), content='1. WebSearchAgent : 2006-2007 시즌에 마이애미 히트에서 가장 많은 득점을 기록한 선수의 정보를 찾아주세요.\n2. WebSearchAgent : 2007-2008 시즌과 2008-2009 시즌의 마이애미 히트 선수들의 총 리바운드 통계를 찾아주세요.\n3. DataAnalystAgent : 2007-2008 시즌과 2008-2009 시즌의 마이애미 히트 총 리바운드 수의 변화율을 계산해주세요.', type='TextMessage'), ToolCallRequestEvent(id='bde225b7-d8ac-4196-90cc-692260a411e6', source='WebSearchAgent', models_usage=RequestUsage(prompt_tokens=275, completi

대화 로그에서 계획 에이전트가 항상 전문 에이전트 이후에 즉시 발언하는 것을 볼 수 있습니다.

```{tip}
각 참가자 에이전트는 각 차례에 한 단계만 수행합니다 (도구 실행, 응답 생성 등).
{py:class}`~autogen_agentchat.agents.AssistantAgent`가 실행해야 하는 모든 도구 실행을 마쳤을 때 {py:class}`~autogen_agentchat.messages.ToolCallSummaryMessage` 반환을 중지할 때까지 반복하기를 원한다면, 마지막 메시지를 확인하고 그것이 {py:class}`~autogen_agentchat.messages.ToolCallSummaryMessage`인 경우 에이전트를 반환하여 그렇게 할 수 있습니다.
```

## 사용자 정의 후보 함수

또 다른 가능한 요구사항은 필터링된 에이전트 목록에서 다음 발언자를 자동으로 선택하는 것일 수 있습니다.
이를 위해 그룹 채팅의 각 차례에서 발언자 선택을 위한 잠재적 에이전트 목록을 필터링하는 사용자 정의 후보 함수로 `candidate_func` 매개변수를 설정할 수 있습니다.

이를 통해 주어진 에이전트 이후에 특정 에이전트 세트로 발언자 선택을 제한할 수 있습니다.

```{note}
`candidate_func`는 `selector_func`가 설정되지 않은 경우에만 유효합니다.
사용자 정의 후보 함수에서 `None` 또는 빈 목록 `[]`을 반환하면 `ValueError`가 발생합니다.
```

In [26]:
def candidate_func(messages: Sequence[BaseAgentEvent | BaseChatMessage]) -> List[str]:
    # keep planning_agent first one to plan out the tasks
    if messages[-1].source == "user":
        return [planning_agent.name]

    # if previous agent is planning_agent and if it explicitly asks for web_search_agent
    # or data_analyst_agent or both (in-case of re-planning or re-assignment of tasks)
    # then return those specific agents
    last_message = messages[-1]
    if last_message.source == planning_agent.name:
        participants = []
        if web_search_agent.name in last_message.to_text():
            participants.append(web_search_agent.name)
        if data_analyst_agent.name in last_message.to_text():
            participants.append(data_analyst_agent.name)
        if participants:
            return participants  # SelectorGroupChat will select from the remaining two agents.

    # we can assume that the task is finished once the web_search_agent
    # and data_analyst_agent have took their turns, thus we send
    # in planning_agent to terminate the chat
    previous_set_of_agents = set(message.source for message in messages)
    if web_search_agent.name in previous_set_of_agents and data_analyst_agent.name in previous_set_of_agents:
        return [planning_agent.name]

    # if no-conditions are met then return all the agents
    return [planning_agent.name, web_search_agent.name, data_analyst_agent.name]


# Reset the previous team and run the chat again with the selector function.
await team.reset()
team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=termination,
    candidate_func=candidate_func,
)

await Console(team.run_stream(task=task))

---------- TextMessage (user) ----------
2006-2007 시즌에 가장 많은 득점을 기록한 마이애미 히트 선수는 누구이며, 2007-2008 시즌과 2008-2009 시즌의 총 리바운드 변화율은 얼마였나요?


---------- TextMessage (PlanningAgent) ----------
1. WebSearchAgent: 2006-2007 시즌에 마이애미 히트에서 가장 많은 득점을 기록한 선수 찾기
2. WebSearchAgent: 2007-2008 시즌과 2008-2009 시즌의 마이애미 히트 팀 총 리바운드 기록 수집하기
3. DataAnalystAgent: 수집된 2007-2008 시즌과 2008-2009 시즌의 총 리바운드 데이터를 바탕으로 변화율 계산하기
---------- ToolCallRequestEvent (WebSearchAgent) ----------
[FunctionCall(id='call_sQ0aTkEH5MlzVC1ti9aSYyqB', arguments='{"query": "2006-2007 Miami Heat top scorer"}', name='search_web_tool'), FunctionCall(id='call_lupyWkM5BD3MKTtLnkdcEAIP', arguments='{"query": "Miami Heat total rebounds 2007-2008 season"}', name='search_web_tool'), FunctionCall(id='call_EVj1z3saRVJwraG6lrucsLnr', arguments='{"query": "Miami Heat total rebounds 2008-2009 season"}', name='search_web_tool')]
---------- ToolCallExecutionEvent (WebSearchAgent) ----------
[FunctionExecutionResult(content='2006-2007 시즌 마이애미 히트 선수들이 득점한 총 득점은 다음과 같습니다:\n        우도니스 하슬렘: 844점\n        드웨인 웨이드: 1397점\n        제임스 포시: 550점\n        ...\n        ', name='search_web_too

TaskResult(messages=[TextMessage(id='c8af67b7-d984-450a-af9e-8e8d88099861', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 15, 6, 9, 12, 853921, tzinfo=datetime.timezone.utc), content='2006-2007 시즌에 가장 많은 득점을 기록한 마이애미 히트 선수는 누구이며, 2007-2008 시즌과 2008-2009 시즌의 총 리바운드 변화율은 얼마였나요?', type='TextMessage'), TextMessage(id='d36fa056-25cb-44b6-ad1f-739a0e05e68b', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=168, completion_tokens=113), metadata={}, created_at=datetime.datetime(2025, 7, 15, 6, 9, 14, 44532, tzinfo=datetime.timezone.utc), content='1. WebSearchAgent: 2006-2007 시즌에 마이애미 히트에서 가장 많은 득점을 기록한 선수 찾기\n2. WebSearchAgent: 2007-2008 시즌과 2008-2009 시즌의 마이애미 히트 팀 총 리바운드 기록 수집하기\n3. DataAnalystAgent: 수집된 2007-2008 시즌과 2008-2009 시즌의 총 리바운드 데이터를 바탕으로 변화율 계산하기', type='TextMessage'), ToolCallRequestEvent(id='ec1648f5-df19-42cd-afec-dee9b35140e4', source='WebSearchAgent', models_usage=RequestUsage(prompt_tokens=268, completion_tokens=90), metad

대화 로그에서 웹 검색 에이전트와 데이터 분석 에이전트가 차례를 가진 후 계획 에이전트가 대화로 돌아가는 것을 볼 수 있으며, 예상대로 작업이 완료되지 않았다는 것을 발견하여 웹 검색 에이전트를 다시 호출하여 리바운드 값을 얻고 데이터 분석 에이전트를 호출하여 백분율 변화를 구했습니다.

## 사용자 피드백

실행 중 사용자 피드백을 제공하기 위해 팀에 {py:class}`~autogen_agentchat.agents.UserProxyAgent`를 추가할 수 있습니다.
{py:class}`~autogen_agentchat.agents.UserProxyAgent`에 대한 자세한 내용은 [휴먼 인 더 루프](./tutorial/human-in-the-loop.ipynb)를 참조하세요.

웹 검색 예제에서 {py:class}`~autogen_agentchat.agents.UserProxyAgent`를 사용하려면 단순히 팀에 추가하고 계획 에이전트가 발언한 후 항상 사용자 피드백을 확인하도록 선택자 함수를 업데이트하면 됩니다.
사용자가 `"APPROVE"`로 응답하면 대화가 계속되고, 그렇지 않으면 사용자가 승인할 때까지 계획 에이전트가 다시 시도합니다.

In [27]:
user_proxy_agent = UserProxyAgent("UserProxyAgent", description="A proxy for the user to approve or disapprove tasks.")


def selector_func_with_user_proxy(messages: Sequence[BaseAgentEvent | BaseChatMessage]) -> str | None:
    if messages[-1].source != planning_agent.name and messages[-1].source != user_proxy_agent.name:
        # Planning agent should be the first to engage when given a new task, or check progress.
        return planning_agent.name
    if messages[-1].source == planning_agent.name:
        if messages[-2].source == user_proxy_agent.name and "APPROVE" in messages[-1].content.upper():  # type: ignore
            # User has approved the plan, proceed to the next agent.
            return None
        # Use the user proxy agent to get the user's approval to proceed.
        return user_proxy_agent.name
    if messages[-1].source == user_proxy_agent.name:
        # If the user does not approve, return to the planning agent.
        if "APPROVE" not in messages[-1].content.upper():  # type: ignore
            return planning_agent.name
    return None


# Reset the previous agents and run the chat again with the user proxy agent and selector function.
await team.reset()
team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent, user_proxy_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    selector_func=selector_func_with_user_proxy,
    allow_repeated_speaker=True,
)

await Console(team.run_stream(task=task))

---------- TextMessage (user) ----------
2006-2007 시즌에 가장 많은 득점을 기록한 마이애미 히트 선수는 누구이며, 2007-2008 시즌과 2008-2009 시즌의 총 리바운드 변화율은 얼마였나요?
---------- TextMessage (PlanningAgent) ----------
1. WebSearchAgent : 2006-2007 시즌 마이애미 히트에서 가장 많은 득점을 기록한 선수 정보를 검색  
2. WebSearchAgent : 2007-2008 시즌과 2008-2009 시즌 마이애미 히트 팀의 총 리바운드 데이터를 수집  
3. DataAnalystAgent : 2007-2008 시즌과 2008-2009 시즌 마이애미 히트 팀 총 리바운드의 변화율을 계산
---------- TextMessage (UserProxyAgent) ----------
approve
---------- ToolCallRequestEvent (WebSearchAgent) ----------
[FunctionCall(id='call_aVAK7UoWvlYaLJynnsudwKgm', arguments='{"query": "2006-2007 season Miami Heat top scorer"}', name='search_web_tool'), FunctionCall(id='call_0jIaFEk2kTM1GuccGMUFcG1S', arguments='{"query": "2007-2008 Miami Heat team total rebounds"}', name='search_web_tool'), FunctionCall(id='call_1vpDi7rVTaljH2uL6skHa5Fq', arguments='{"query": "2008-2009 Miami Heat team total rebounds"}', name='search_web_tool')]
---------- ToolCallExecutionEvent (WebSearchAgent) -----

TaskResult(messages=[TextMessage(id='62660d57-10c3-48bf-aeb8-138957308c0c', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 15, 6, 9, 59, 468136, tzinfo=datetime.timezone.utc), content='2006-2007 시즌에 가장 많은 득점을 기록한 마이애미 히트 선수는 누구이며, 2007-2008 시즌과 2008-2009 시즌의 총 리바운드 변화율은 얼마였나요?', type='TextMessage'), TextMessage(id='f8b17b40-e831-46fa-81f5-b21fc94b4188', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=168, completion_tokens=112), metadata={}, created_at=datetime.datetime(2025, 7, 15, 6, 10, 0, 627127, tzinfo=datetime.timezone.utc), content='1. WebSearchAgent : 2006-2007 시즌 마이애미 히트에서 가장 많은 득점을 기록한 선수 정보를 검색  \n2. WebSearchAgent : 2007-2008 시즌과 2008-2009 시즌 마이애미 히트 팀의 총 리바운드 데이터를 수집  \n3. DataAnalystAgent : 2007-2008 시즌과 2008-2009 시즌 마이애미 히트 팀 총 리바운드의 변화율을 계산', type='TextMessage'), UserInputRequestedEvent(id='6f663f01-3024-423e-a040-2394fdf5cc2c', source='UserProxyAgent', models_usage=None, metadata={}, created_at=datetime.datetime(202

이제 사용자의 피드백이 대화 흐름에 통합되고, 사용자가 계획 에이전트의 결정을 승인하거나 거부할 수 있습니다.

## 추론 모델 사용

지금까지 예제에서는 `gpt-4o` 모델을 사용했습니다. `gpt-4o`와 `gemini-1.5-flash` 같은 모델은 지시사항을 잘 따르므로, 팀의 선택자 프롬프트와 각 에이전트의 시스템 메시지에 상대적으로 자세한 지시사항을 포함하여 그들의 행동을 안내할 수 있습니다.

하지만 `o3-mini`와 같은 추론 모델을 사용하는 경우, 선택자 프롬프트와 시스템 메시지를 가능한 한 간단하고 요점에 맞게 유지해야 합니다. 추론 모델은 이미 제공된 컨텍스트를 바탕으로 자체 지시사항을 도출하는 데 뛰어나기 때문입니다.

이것은 또한 추론 모델을 사용하는 {py:class}`~autogen_agentchat.teams.SelectorGroupChat`가 자체적으로 그 작업을 수행할 수 있으므로 더 이상 작업을 나누기 위한 계획 에이전트가 필요하지 않다는 의미입니다.

다음 예제에서는 에이전트와 팀의 모델로 `o3-mini`를 사용하고, 계획 에이전트는 사용하지 않겠습니다.
또한 선택자 프롬프트와 시스템 메시지를 가능한 한 간단하게 유지하겠습니다.

In [28]:
reasoning_deployment_name = os.getenv("AZURE_OPENAI_REASONING_DEPLOYMENT_NAME") 

azure_openai_chat_completion_client = AzureOpenAIChatCompletionClient(
            model=reasoning_deployment_name,
            azure_endpoint=azure_endpoint,
            api_version="2024-12-01-preview",
            api_key=api_key
)


model_client = azure_openai_chat_completion_client

web_search_agent = AssistantAgent(
    "WebSearchAgent",
    description="An agent for searching information on the web.",
    tools=[search_web_tool],
    model_client=model_client,
    system_message="""Use web search tool to find information.""",
)

data_analyst_agent = AssistantAgent(
    "DataAnalystAgent",
    description="An agent for performing calculations.",
    model_client=model_client,
    tools=[percentage_change_tool],
    system_message="""Use tool to perform calculation. If you have not seen the data, ask for it.""",
)

user_proxy_agent = UserProxyAgent(
    "UserProxyAgent",
    description="A user to approve or disapprove tasks.",
)

selector_prompt = """Select an agent to perform task.

{roles}

Current conversation context:
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
When the task is complete, let the user approve or disapprove the task.
"""

team = SelectorGroupChat(
    [web_search_agent, data_analyst_agent, user_proxy_agent],
    model_client=model_client,
    termination_condition=termination,  # Use the same termination condition as before.
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,
)

In [29]:
await Console(team.run_stream(task=task))

---------- TextMessage (user) ----------
2006-2007 시즌에 가장 많은 득점을 기록한 마이애미 히트 선수는 누구이며, 2007-2008 시즌과 2008-2009 시즌의 총 리바운드 변화율은 얼마였나요?
---------- ToolCallRequestEvent (WebSearchAgent) ----------
[FunctionCall(id='call_DLbKUiH4saBVt13XvOIWOCx9', arguments='{"query": "2006-2007 시즌에 가장 많은 득점을 기록한 마이애미 히트 선수 그리고 2007-2008 시즌과 2008-2009 시즌의 총 리바운드 변화율은 얼마인가? \\"2006-07 Miami Heat top scorer and rebound percentage change 2007-2008 vs 2008-2009\\""}', name='search_web_tool')]
---------- ToolCallExecutionEvent (WebSearchAgent) ----------
[FunctionExecutionResult(content='2006-2007 시즌 마이애미 히트 선수들이 득점한 총 득점은 다음과 같습니다:\n        우도니스 하슬렘: 844점\n        드웨인 웨이드: 1397점\n        제임스 포시: 550점\n        ...\n        ', name='search_web_tool', call_id='call_DLbKUiH4saBVt13XvOIWOCx9', is_error=False)]
---------- ToolCallSummaryMessage (WebSearchAgent) ----------
2006-2007 시즌 마이애미 히트 선수들이 득점한 총 득점은 다음과 같습니다:
        우도니스 하슬렘: 844점
        드웨인 웨이드: 1397점
        제임스 포시: 550점
        ...
        
---------- T

Model failed to select a speaker after 3, using the previous speaker.


---------- TextMessage (DataAnalystAgent) ----------
2006-2007 시즌 마이애미 히트의 최고 득점 선수는 드웨인 웨이드입니다. 또한, 드웨인 웨이드는 2007-2008 시즌에 214개의 리바운드를, 2008-2009 시즌에는 398개의 리바운드를 기록하여, 두 시즌 간의 리바운드 변화율은 약 85.98%로 나타났습니다.
---------- TextMessage (DataAnalystAgent) ----------
2006-2007 시즌 마이애미 히트의 최고 득점 선수는 드웨인 웨이드입니다. 또한, 2007-2008 시즌에 드웨인 웨이드는 214개의 리바운드를, 2008-2009 시즌에는 398개의 리바운드를 기록했습니다. 이로 인해 두 시즌 간 리바운드 변화율은 약 85.98%입니다.
---------- TextMessage (DataAnalystAgent) ----------
2006-2007 시즌 마이애미 히트의 최고 득점 선수는 드웨인 웨이드입니다. 또한, 웨이드는 2007-2008 시즌에 214개의 리바운드를, 2008-2009 시즌에는 398개의 리바운드를 기록했으며, 두 시즌 간 리바운드 변화율은 약 85.98%입니다.
---------- TextMessage (DataAnalystAgent) ----------
2006-2007 시즌 마이애미 히트의 최고 득점 선수는 드웨인 웨이드입니다. 또한, 웨이드는 2007-2008 시즌에 214개의 리바운드를, 2008-2009 시즌에는 398개의 리바운드를 기록했으며, 두 시즌 간 리바운드 변화율은 약 85.98%입니다.
---------- TextMessage (DataAnalystAgent) ----------
2006-2007 시즌 마이애미 히트의 최고 득점 선수는 드웨인 웨이드입니다. 또한, 웨이드는 2007-2008 시즌에 214개의 리바운드를 기록했고, 2008-2009 시즌에는 398개의 리바운드를 기록했습니다. 이를 바탕으로 두 시즌 간 리바운드

Model failed to select a speaker after 3, using the previous speaker.


---------- TextMessage (DataAnalystAgent) ----------
2006-2007 시즌 마이애미 히트의 최고 득점 선수는 드웨인 웨이드이며, 2007-2008 시즌에 웨이드는 214개의 리바운드를 기록했습니다. 이후, 2008-2009 시즌에는 398개의 리바운드를 기록하여 두 시즌 간 리바운드 변화율이 약 85.98%임을 알 수 있습니다.
---------- TextMessage (DataAnalystAgent) ----------
2006-2007 시즌 마이애미 히트의 최고 득점 선수는 드웨인 웨이드입니다. 또한, 웨이드는 2007-2008 시즌에 214개의 리바운드를 기록했고, 2008-2009 시즌에는 398개의 리바운드를 기록했습니다. 이를 바탕으로 두 시즌 간 리바운드 변화율은 약 85.98%로 계산됩니다.


TaskResult(messages=[TextMessage(id='745feb01-53a8-49af-804a-d30a54cd384a', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 15, 6, 10, 20, 260652, tzinfo=datetime.timezone.utc), content='2006-2007 시즌에 가장 많은 득점을 기록한 마이애미 히트 선수는 누구이며, 2007-2008 시즌과 2008-2009 시즌의 총 리바운드 변화율은 얼마였나요?', type='TextMessage'), ToolCallRequestEvent(id='d7921caf-381c-4762-8c60-1f78e697204f', source='WebSearchAgent', models_usage=RequestUsage(prompt_tokens=110, completion_tokens=292), metadata={}, created_at=datetime.datetime(2025, 7, 15, 6, 10, 29, 220844, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_DLbKUiH4saBVt13XvOIWOCx9', arguments='{"query": "2006-2007 시즌에 가장 많은 득점을 기록한 마이애미 히트 선수 그리고 2007-2008 시즌과 2008-2009 시즌의 총 리바운드 변화율은 얼마인가? \\"2006-07 Miami Heat top scorer and rebound percentage change 2007-2008 vs 2008-2009\\""}', name='search_web_tool')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='c0fa0bac-8839-4223-800f-f32a68b4608d', source='WebSe

```{tip}
추론 모델을 프롬프트하는 방법에 대한 더 많은 지침은 [OpenAI의 O1 및 O3-mini 추론 모델을 위한 프롬프트 엔지니어링](https://techcommunity.microsoft.com/blog/azure-ai-services-blog/prompt-engineering-for-openai%E2%80%99s-o1-and-o3-mini-reasoning-models/4374010)에 대한 Azure AI Services 블로그를 참조하세요.
```